In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from collections import Counter

In [2]:
user_df = pd.read_excel('../Data/Visitors Preference Dataset.xlsx')
places_df = pd.read_csv('../Data/Places Dataset Classified Reviews Cleaned Combined.csv')

In [3]:
places_df = places_df[['name', 'rating' ,'negetive_rate']]
places_df.head()

,name,rating,negetive_rate
0,Arugam Bay Beach,4.800000,0.0
1,Mirissa Beach,4.600000,0.0
2,Weligama Beach (surf and stay),4.400000,0.0
3,Ahangama,4.459437,0.2
4,Hikkaduwa Beach,4.700000,0.0


In [4]:
def generate_recommended_indexes(size=50, max_index=400):
    random_indexes = np.random.choice(np.arange(0, max_index + 1), size=size, replace=True)
    unique_indexes = set(random_indexes)
    confidence_scores = np.random.rand(len(unique_indexes))
    df_recommended = pd.DataFrame({
        'Index': list(unique_indexes),
        'Confidence': confidence_scores
    })
    
    return df_recommended

df_recommended_indexes = generate_recommended_indexes(size=50, max_index=400)
df_recommended_indexes.head()

,Index,Confidence
0,385,0.643177
1,257,0.947781
2,255,0.502589
3,390,0.677127
4,6,0.387263


In [5]:
import pandas as pd

def find_places_by_index(df_recommended_indexes, places_df):
    # Extract the indexes and confidence scores from the provided DataFrame
    indexes = df_recommended_indexes["Index"].tolist()
    confidence_scores = df_recommended_indexes["Confidence"].tolist()
    
    # Retrieve places based on indexes
    # Convert indexes to integer type if necessary (for safety)
    indexes = list(map(int, indexes))
    places = [places_df.iloc[index] for index in indexes]
    
    # Create a DataFrame for the retrieved places
    df_places = pd.DataFrame(places).reset_index(drop=True)
    
    # Ensure the places DataFrame index matches the order of the provided indexes
    df_places.index = indexes
    
    # Create a DataFrame for the confidence scores
    df_confidence = pd.DataFrame({
        'Confidence': confidence_scores
    }, index=indexes)
    
    # Merge the places DataFrame with the confidence scores DataFrame
    df_places_with_confidence = df_places.join(df_confidence, how='left')
    
    return df_places_with_confidence

# Example usage:
df_places_2 = find_places_by_index(df_recommended_indexes, places_df)
df_places_2.shape


(47, 4)

In [6]:
def check_negetive_rate(places_df, user_df):
    places = []
    for index, row in places_df.iterrows():
        if row['negetive_rate'] <= 0.2:
            places.append(row)
    return pd.DataFrame(places)

df_places_3 = check_negetive_rate(df_places_2, user_df)
df_places_3.shape

(46, 4)

In [7]:
def sort_by_confidence_then_rating(df_places):
    df_places_sorted = df_places.sort_values(by=['Confidence', 'rating'], ascending=False)
    return df_places_sorted

df_places_4 = sort_by_confidence_then_rating(df_places_3)
df_places_4.head(5)

,name,rating,negetive_rate,Confidence
172,Dehiwala Zoological Gardens,4.4,0.0,0.970590
72,Madu Ganga,4.6,0.0,0.966611
276,Hatadage,4.5,0.0,0.956219
257,Monument of Victory,4.3,0.2,0.947781
40,Dutch Museum,3.7,0.0,0.929991
